In [1]:
import os
import polars as pl
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from neo4j import GraphDatabase
from langchain.agents import create_openai_functions_agent, Tool, AgentExecutor
from langchain import hub
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from graph_qa.cypher import GraphCypherQAChain
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [24]:
HOSPITAL_QA_MODEL = os.getenv("HOSPITAL_QA_MODEL")
HOSPITAL_CYPHER_MODEL = os.getenv("HOSPITAL_CYPHER_MODEL")

graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

graph.refresh_schema()

In [31]:
question = "Who is the oldest patient and how old are they?".lower().strip()
cypher_query = f'MATCH (q:Question) WHfERE q.question = "{question}" RETURN q;'


graph.query(cypher_query)

ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'WHfERE': expected a graph pattern, 'FOREACH', ',', 'CALL', 'CREATE', 'LOAD CSV', 'DELETE', 'DETACH', 'FINISH', 'INSERT', 'MATCH', 'MERGE', 'NODETACH', 'OPTIONAL', 'REMOVE', 'RETURN', 'SET', 'UNION', 'UNWIND', 'USE', 'USING', 'WHERE', 'WITH' or <EOF> (line 1, column 20 (offset: 19))
"MATCH (q:Question) WHfERE q.question = "who is the oldest patient and how old are they?" RETURN q;"
                    ^}

In [3]:
HOSPITAL_CYPHER_MODEL

'gpt-3.5-turbo-0125'

#### Few shot Cypher RAG
TODO:
- Create CSV with example questions and answers [X]
- Add to initial ETL [X]
- Create Questions node with properties "question" and "cypher" [X]
- Write function that creates Question nodes from a question and corresponding cypher query
- Need to be able to embed questions on the fly
- Streamlit app to add questions to the db
- Update langchain version and refactor

In [9]:
questions = [
    "Who is the oldest patient and how old are they?",
    "Which physician has billed the least to Cigna",
    "How many non-emergency patients in North Carolina have written reviews?",
]

cypher_answers = [
    """
    MATCH (p:Payer)<-[c:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)
    WHERE p.name = 'Cigna'
    RETURN phy.name AS physician_name, SUM(c.billing_amount) AS total_billed
    ORDER BY total_billed
    LIMIT 1
    """,
    """
    MATCH (p:Payer)<-[c:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)
    WHERE p.name = 'Cigna'
    RETURN phy.name AS physician_name, SUM(c.billing_amount) AS total_billed
    ORDER BY total_billed
    LIMIT 1
    """,
    """
    match (r:Review)<-[:WRITES]-(v:Visit)-[:AT]->(h:Hospital)
    where h.state_name = 'NC' and v.admission_type <> 'Emergency'
    return count(*)
    """
]

data = pl.DataFrame({
    "question":questions,
    "cypher":cypher_answers
})



'''# Which state had the largest percent increase in Cigna visits
# from 2022 to 2023?
MATCH (h:Hospital)<-[:AT]-(v:Visit)-[:COVERED_BY]->(p:Payer)
WHERE p.name = 'Cigna' AND v.admission_date >= '2022-01-01' AND
v.admission_date < '2024-01-01'
WITH h.state_name AS state, COUNT(v) AS visit_count,
     SUM(CASE WHEN v.admission_date >= '2022-01-01' AND
     v.admission_date < '2023-01-01' THEN 1 ELSE 0 END) AS count_2022,
     SUM(CASE WHEN v.admission_date >= '2023-01-01' AND
     v.admission_date < '2024-01-01' THEN 1 ELSE 0 END) AS count_2023
WITH state, visit_count, count_2022, count_2023,
     (toFloat(count_2023) - toFloat(count_2022)) / toFloat(count_2022) * 100
     AS percent_increase
RETURN state, percent_increase
ORDER BY percent_increase DESC
LIMIT 1
'''

data.write_csv("../data/example_cypher.csv")

In [10]:
def add_document_node(uri, user, password, question, cypher, embedding):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    with driver.session() as session:
        session.write_transaction(create_document_node, question, cypher, embedding)
    driver.close()

def create_document_node(tx, question, cypher, embedding):
    query = (
        "CREATE (d:Document {question: $question, cypher: $cypher, embedding: $embedding})"
    )
    tx.run(query, question=question, cypher=cypher, embedding=embedding)

In [23]:
neo4j_question_vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="questions",
    node_label="Question",
    text_node_properties=[
        "question",
    ],
    text_node_property="question",
    embedding_node_property="embedding",
)

neo4j_question_vector_index = Neo4jVector.from_existing_index(
    embedding=OpenAIEmbeddings(),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="questions",
    text_node_property="question",
)
neo4j_question_vector_index

In [27]:
neo4j_question_vector_index.add_texts(
    texts=[
        "Which physician has billed the most in Florida?"
    ],
    metadatas=[
        {
            "cypher":"""
            MATCH (p:Payer)<-[c:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)<-[:EMPLOYS]-(h:Hospital)
            WHERE h.state_name = 'FL'
            RETURN phy.name AS physician_name, SUM(c.billing_amount) AS total_billed
            ORDER BY total_billed DESC
            LIMIT 1"""
        }
    ]
)

['403d3999e8fab4a8bf25cb8e73dd071d']

In [32]:
neo4j_question_vector_index.similarity_search("Which doctors have billed the most in Texas? Show me the top 5.", k=4)

Failed to write data to connection ResolvedIPv4Address(('34.66.78.163', 7687)) (ResolvedIPv4Address(('34.66.78.163', 7687)))
Failed to write data to connection IPv4Address(('d3a109b6.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.66.78.163', 7687)))


[Document(metadata={'cypher': "MATCH (p:Payer)<-[c:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)\n    WHERE p.name = 'Cigna'\n    RETURN phy.name AS physician_name, SUM(c.billing_amount) AS total_billed\n    ORDER BY total_billed\n    LIMIT 1"}, page_content='which physician has billed the least to cigna?'),
 Document(metadata={'cypher': "MATCH (p:Payer)<-[c:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)\n    WHERE p.name = 'Cigna'\n    RETURN phy.name AS physician_name, SUM(c.billing_amount) AS total_billed\n    ORDER BY total_billed\n    LIMIT 1"}, page_content='who is the oldest patient and how old are they?'),
 Document(metadata={'cypher': "match (r:Review)<-[:WRITES]-(v:Visit)-[:AT]->(h:Hospital)\n    where h.state_name = 'NC' and v.admission_type <> 'Emergency'\n    return count(*)"}, page_content='how many non-emergency patients in north carolina have written reviews?')]

#### Cypher chain

In [9]:
cypher_example_retriever = neo4j_question_vector_index.as_retriever(search_kwargs={'k': 4})

In [15]:
cypher_generation_template = """
Task:
Generate Cypher query for a Neo4j graph database.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.

Schema:
{schema}

Note:
Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything other than
for you to construct a Cypher statement. Do not include any text except
the generated Cypher statement. Make sure the direction of the relationship is
correct in your queries. Make sure you alias both entities and relationships
properly. Do not run any queries that would add to or delete from
the database. Make sure to alias all statements that follow as with
statement (e.g. WITH v as visit, c.billing_amount as billing_amount)
If you need to divide numbers, make sure to
filter the denominator to be non zero.

Example queries for this schema:
{example_queries}

String category values:
Test results are one of: 'Inconclusive', 'Normal', 'Abnormal'
Visit statuses are one of: 'OPEN', 'DISCHARGED'
Admission Types are one of: 'Elective', 'Emergency', 'Urgent'
Payer names are one of: 'Cigna', 'Blue Cross', 'UnitedHealthcare', 'Medicare',
'Aetna'

A visit is considered open if its status is 'OPEN' and the discharge date is
missing.

Make sure to use IS NULL or IS NOT NULL when analyzing missing properties.
Never return embedding properties in your queries. You must never include the
statement "GROUP BY" in your query. Make sure to alias all statements that
follow as with statement (e.g. WITH v as visit, c.billing_amount as
billing_amount)
If you need to divide numbers, make sure to filter the denominator to be non
zero.

Use state abbreviations instead of their full name. For example, you should
change "Texas" to "TX", "Colorado" to "CO", "North Carolina" to "NC", and so on.

The question is:
{question}
"""

cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "example_queries", "question"], template=cypher_generation_template
)

qa_generation_template = """You are an assistant that takes the results
from a Neo4j Cypher query and forms a human-readable response. The
query results section contains the results of a Cypher query that was
generated based on a users natural language question. The provided
information is authoritative, you must never doubt it or try to use
your internal knowledge to correct it. Make the answer sound like a
response to the question.

Query Results:
{context}

Question:
{question}

If the provided information is empty, say you don't know the answer.
Empty information looks like this: []

If the information is not empty, you must provide an answer using the
results. If the question involves a time duration, assume the query
results are in units of days unless otherwise specified.

When names are provided in the query results, such as hospital names,
beware  of any names that have commas or other punctuation in them.
For instance, 'Jones, Brown and Murray' is a single hospital name,
not multiple hospitals. Make sure you return any list of names in
a way that isn't ambiguous and allows someone to tell what the full
names are.

Never say you don't have the right information if there is data in
the query results. Make sure to show all the relevant query results
if you're asked.

Helpful Answer:
"""

qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template
)

hospital_cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    qa_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    cypher_example_retriever=cypher_example_retriever,
    graph=graph,
    verbose=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    validate_cypher=True,
    top_k=100,
)

In [16]:
hospital_cypher_chain.invoke("Which doctors have billed the most in Texas? Show me the top 5.")



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:Payer)<-[c:COVERED_BY]-(v:Visit)-[t:TREATS]-(phy:Physician)<-[:EMPLOYS]-(h:Hospital)
WHERE h.state_name = 'TX'
RETURN phy.name AS physician_name, SUM(c.billing_amount) AS total_billed
ORDER BY total_billed DESC
LIMIT 5

Full Context:
[{'physician_name': 'Tracy Garza', 'total_billed': 2603139.78504043}, {'physician_name': 'Lauren Williams', 'total_billed': 2541214.8581720726}, {'physician_name': 'Sara Lopez', 'total_billed': 2498166.8090435355}, {'physician_name': 'Carla Miller', 'total_billed': 2454530.8802457894}, {'physician_name': 'Sean Best', 'total_billed': 2439745.2936023297}]

> Finished chain.


{'query': 'Which doctors have billed the most in Texas? Show me the top 5.',
 'result': 'The top 5 doctors who have billed the most in Texas are:\n\n1. **Tracy Garza** - $2,603,139.79\n2. **Lauren Williams** - $2,541,214.86\n3. **Sara Lopez** - $2,498,166.81\n4. **Carla Miller** - $2,454,530.88\n5. **Sean Best** - $2,439,745.29\n\nThese physicians have significantly contributed to the total billed amounts in the state.'}

#### Review chain

In [19]:
neo4j_vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="reviews",
    node_label="Review",
    text_node_properties=[
        "physician_name",
        "patient_name",
        "text",
        "hospital_name",
    ],
    embedding_node_property="embedding",
)

review_template = """Your job is to use patient
reviews to answer questions about their experience at a hospital. Use
the following context to answer questions. Be as detailed as possible, but
don't make up any information that's not from the context. If you don't know
an answer, say you don't know.
{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_template
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [review_system_prompt, review_human_prompt]

review_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

reviews_vector_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model=HOSPITAL_QA_MODEL, temperature=0),
    chain_type="stuff",
    retriever=neo4j_vector_index.as_retriever(k=12),
)
reviews_vector_chain.combine_documents_chain.llm_chain.prompt = review_prompt


#### Agent

In [11]:
HOSPITAL_AGENT_MODEL = os.getenv("HOSPITAL_AGENT_MODEL")

hospital_agent_prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [
    Tool(
        name="Experiences",
        func=reviews_vector_chain.invoke,
        description="""Useful when you need to answer questions
        about patient experiences, feelings, or any other qualitative
        question that could be answered about a patient using semantic
        search. Not useful for answering objective questions that involve
        counting, percentages, aggregations, or listing facts. Use the
        entire prompt as input to the tool. For instance, if the prompt is
        "Are patients satisfied with their care?", the input should be
        "Are patients satisfied with their care?".
        """,
    ),
    Tool(
        name="Graph",
        func=hospital_cypher_chain.invoke,
        description="""Useful for answering questions about patients,
        physicians, hospitals, insurance payers, patient review
        statistics, and hospital visit details. Use the entire prompt as
        input to the tool. For instance, if the prompt is "How many visits
        have there been?", the input should be "How many visits have
        there been?".
        """,
    ),
]

chat_model = ChatOpenAI(
    model=HOSPITAL_AGENT_MODEL,
    temperature=0,
)

hospital_rag_agent = create_openai_functions_agent(
    llm=chat_model,
    prompt=hospital_agent_prompt,
    tools=tools,
)

hospital_rag_agent_executor = AgentExecutor(
    agent=hospital_rag_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

In [17]:
hospital_rag_agent_executor.invoke({"input":"What is the average duration in days for closed emergency visits?"})



> Entering new AgentExecutor chain...

Invoking: `Graph` with `What is the average duration in days for closed emergency visits?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visit)-[:AT]->(h:Hospital)
WHERE v.status = 'DISCHARGED' AND v.admission_type = 'Emergency'
WITH v, duration.between(date(v.admission_date), date(v.discharge_date)).days AS duration
RETURN AVG(duration) AS average_duration_days
Full Context:
[{'average_duration_days': 15.117444409646081}]

> Finished chain.
{'query': 'What is the average duration in days for closed emergency visits?', 'result': 'The average duration in days for closed emergency visits is approximately 15.12 days.'}The average duration in days for closed emergency visits is approximately 15.12 days.

> Finished chain.


{'input': 'What is the average duration in days for closed emergency visits?',
 'output': 'The average duration in days for closed emergency visits is approximately 15.12 days.',
 'intermediate_steps': [(AgentActionMessageLog(tool='Graph', tool_input='What is the average duration in days for closed emergency visits?', log='\nInvoking: `Graph` with `What is the average duration in days for closed emergency visits?`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"What is the average duration in days for closed emergency visits?"}', 'name': 'Graph'}})]),
   {'query': 'What is the average duration in days for closed emergency visits?',
    'result': 'The average duration in days for closed emergency visits is approximately 15.12 days.'})]}